In [54]:
%load_ext autoreload
%autoreload 2

import rosbag
import rospy
import os
import glob
import cv2
import pickle 
import numpy as np
import matplotlib.pyplot as plt
import json
from cv_bridge import CvBridge
import tf.transformations as tftr
from matplotlib import cm
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable, get_cmap
from sonar_img_utilities import draw_sonar_msg, find_nearest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
save_loc = "/home/marc/Documents/neusis/data/test_data"
cfg_file = os.path.join(save_loc, "Config.json")
sim_bag = "/home/marc/Documents/sonar_reconstruction/bagfiles/sim_sonar.bag"

cfg_dict = {"name": "SimImagingSonar", 
            "agents":[
    {
    "agent_name": "auv0", 
     "agent_type": "HoveringAUV", 
     "sensors": 
        [{
        "sensor_type": "ImagingSonar", 
        "configuration": {}
        }]
    }
        ]}

json_object = json.dumps(cfg_dict)
with open(cfg_file, "w") as outfile:
    outfile.write(json_object) 

In [65]:
tf_topic = "/tf"
bag = rosbag.Bag(open(sim_bag, "rb"))

tf_array = []
for topic, msg, tt in bag.read_messages(tf_topic):
        tfs = msg.transforms[0]
        translation = [tfs.transform.translation.x, tfs.transform.translation.y, tfs.transform.translation.z]
        rotation = [tfs.transform.rotation.x, tfs.transform.rotation.y, tfs.transform.rotation.z, tfs.transform.rotation.w]
        pose_matrix = tftr.compose_matrix(translate=translation, angles=tftr.euler_from_quaternion(rotation))
        tf_array.append([tt.to_nsec(), pose_matrix])

time_array = np.array(tf_array)[:, 0]

/home/marc/miniconda3/envs/neusis/lib/python3.7/site-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


In [72]:
sonar_topic = "/sim_oculus/oculus/sonar_image"
bridge = CvBridge()
bag = rosbag.Bag(open(sim_bag, "rb"))

cfg_file_flag = False

cnt = 0
for topic, msg, tt in bag.read_messages(sonar_topic):
    if not cfg_file_flag:
        with open(cfg_file, 'r') as openfile:
                # Reading from json file
                cfg = json.load(openfile)
        d = {}
        d["RangeBins"] = len(msg.ranges)
        d["AzimuthBins"] = len(msg.azimuth_angles)
        d["RangeMin"] = min(msg.ranges)
        d["RangeMax"] = max(msg.ranges)
        d["Elevation"] = np.rad2deg(msg.elevation_beamwidth)
        d["Azimuth"] = np.rad2deg(max(msg.azimuth_angles))
        cfg['agents'][0]['sensors'][0]['configuration'] = d
        json_object = json.dumps(cfg)
        with open(cfg_file, "w") as outfile:
                outfile.write(json_object)

        
        cfg_file_flag = True
        print('wrote file')

    msg_dict = {}
    sonar_img, ranges, angles = draw_sonar_msg(msg, return_processed=False, trim=False)
    sonar_img = sonar_img.astype(np.uint32)
    msg_dict["ImagingSonar"] = sonar_img

    # find closest pose in time:
    val, idx = find_nearest(time_array, tt.to_nsec())
    pose = tf_array[idx][1]
    msg_dict["PoseSensor"] = pose

    fname = f"{str(cnt).zfill(5)}.pkl"
    full_fname = os.path.join(save_loc, "Data", fname)
    with open(full_fname, 'wb') as handle:
        pickle.dump(msg_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


    cnt += 1
    print(f"Wrote {fname}", end='\r')

wrote file
